In [2]:
import tensorflow as tf
from utils import save 
import numpy as np
from models._24_net import calib_24Net
from utils.utilities import batch_selection
import sys
from utils.params import *
import time
from utils import save

In [3]:
batch_size = 256
epochs = 2000
lr = [1.2e-2, 1e-2, 0.9e-2]
bias_init = [0.0]
reg = [0.0, 1e-4]
_24_calib_batch = batch_selection()

def valid_correct_percentage(pred, truth, top_numb=3):
    assert(truth.shape[0] == pred.shape[0])
    top_score = [arr.argsort()[-top_numb:][::-1] for arr in pred]
    score_list = [1 if truth[i] in top_score[i] else 0 for i in range(truth.shape[0])]
    percentage = np.sum(score_list) / truth.shape[0]
    return percentage

In [4]:
calibrate_db_24 = save.loader('calibrate_db_24_long.txt')
calibrate_db_24 = calibrate_db_24[:int(len(calibrate_db_24) / batch_size) * batch_size]
inputs_calib_24 = np.zeros((len(calibrate_db_24), 24, 24, 3))
targets_calib_24 = np.zeros((len(calibrate_db_24), 45), dtype=np.int32)

for i in range(len(calibrate_db_24)):
    inputs_calib_24[i,:] = calibrate_db_24[i][0]
    targets_calib_24[i,calibrate_db_24[i][1]] = 1

Loaded from D:\programing project\python project\trying stuffpickle/


In [5]:
inputs = np.zeros((batch_size, 24, 24, 3))
targets = np.zeros((batch_size, 45), dtype=np.int32)

In [6]:
result_list = save.loader('calibrate_db_24_test.txt')
valid_input = np.concatenate([result_list[i][0][np.newaxis] for i in range(len(result_list))])
valid_target = np.asarray([result_list[i][1] for i in range(len(result_list))])


Loaded from D:\programing project\python project\trying stuffpickle/


In [10]:
for r in reg:
    for b in bias_init:
        for l in lr:
            record_l = l
            print('**Bias_init:{}   Learning_rate:{}   Regularization:{}**'.format(b,l,r))

            max_i = 0
            correct_pnt_top3 = 0
            correct_pnt_top1 = 0
            correct_pnt_top3_max = 0
            correct_pnt_top1_max = 0
            bad_cnt = 0

            start = time.time()
            g=tf.Graph()
            with g.as_default():
                input_24_node = tf.placeholder(tf.float32, [None, 24, 24, 3])
                target_24_node = tf.placeholder(tf.float32, [None, 45])
                keep_prob = tf.placeholder(tf.float32)
                _calib_24 = calib_24Net(input_24_node, target_24_node, keep_prob, l, b, r)

            sess = tf.InteractiveSession(config=tf.ConfigProto(
              allow_soft_placement=True, log_device_placement=True), graph=g)
            sess.run(tf.global_variables_initializer())
            train_writer = tf.summary.FileWriter('./logs/train/Calib_24/bias_init={},lr={},reg={}'.format(b,l,r))

            for i in range(epochs):

                if (i+1) % 5 == 0:
                    pred_target = _calib_24.prediction.eval(feed_dict={input_24_node:valid_input,keep_prob:1.0})
                    correct_pnt_top3 = valid_correct_percentage(pred_target, valid_target, 3)
                    correct_pnt_top1 = valid_correct_percentage(pred_target, valid_target, 1)

                    if correct_pnt_top3 > correct_pnt_top3_max:
                        max_i = i + 1
                        correct_pnt_top3_max = correct_pnt_top3
                        correct_pnt_top1_max = correct_pnt_top1
                        bad_cnt = 0
                        saver = tf.train.Saver()
                        saver.save(sess, p_model_dir + 'calib_24/max/_Net24_b{}_lr{}_reg{}_drop{}.ckpt'.format(b,record_l,r,0.5))

                    else:
                        bad_cnt += 1

                if i > 30 and bad_cnt % 5 == 0:
                    l = l * .98
                    
                mean_var_list = np.zeros((4,64))
                
                if bad_cnt >= 80:
                    break

                for batch_i in range(int(len(calibrate_db_24) / batch_size)):
                    iters = i*int(len(calibrate_db_24) / batch_size) + batch_i
                    inputs, targets = _24_calib_batch.next_batch(inputs_calib_24, targets_calib_24, batch_size)
                    feed_train = {
                        input_24_node:inputs,
                        target_24_node:targets,
                        keep_prob:.5
                    }
                    feed_loss = {
                        input_24_node:inputs,
                        target_24_node:targets,
                        keep_prob:1.0

                    }

                    cost = _calib_24.loss.eval(feed_loss)
                    sys.stdout.write('\rEpoch:' + str(i+1) + \
                                     '    Iters:' + str(iters+1) + '/' + str(int(len(calibrate_db_24) / batch_size)*epochs) + \
                                     '    Loss:' + str(cost)[:8] + \
                                     '    Cur_Accuracy:' + str(correct_pnt_top1*100)[:5] + '%/' + str(correct_pnt_top3*100)[:5] + '%' + \
                                     '    Max_Acc/Epo:' + str(correct_pnt_top1_max*100)[:5] + '%/' + str(correct_pnt_top3_max*100)[:5] + \
                                                          '%/' + str(max_i) + \
                                     '    Unimprove_Count:' + str(bad_cnt))

                    _calib_24.train_step.run(feed_train)
            end = time.time()
            print('\nTrained Time:{} mins\n'.format(str((end-start)/60.0)[:5]))

**Bias_init:0.0   Learning_rate:0.012   Regularization:0.0**
Epoch:564    Iters:43992/156000    Loss:1.74056    Cur_Accuracy:36.88%/66.0%    Max_Acc/Epo:35.44%/71.55%/165    Unimprove_Count:79
Trained Time:22.62 mins

**Bias_init:0.0   Learning_rate:0.01   Regularization:0.0**
Epoch:589    Iters:45942/156000    Loss:2.57734    Cur_Accuracy:19.77%/46.66%    Max_Acc/Epo:20.55%/51.44%/190    Unimprove_Count:79
Trained Time:23.03 mins

**Bias_init:0.0   Learning_rate:0.009   Regularization:0.0**
Epoch:639    Iters:49842/156000    Loss:2.42075    Cur_Accuracy:16.0%/42.88%    Max_Acc/Epo:20.33%/51.77%/240    Unimprove_Count:79
Trained Time:24.84 mins

**Bias_init:0.0   Learning_rate:0.012   Regularization:0.0001**
Epoch:684    Iters:53352/156000    Loss:2.55631    Cur_Accuracy:18.11%/53.11%    Max_Acc/Epo:21.33%/55.11%/285    Unimprove_Count:79
Trained Time:26.62 mins

**Bias_init:0.0   Learning_rate:0.01   Regularization:0.0001**
Epoch:454    Iters:35412/156000    Loss:1.3263    Cur_Accurac